In [1]:
import pandas as pd
import numpy as np
import pickle

# from crossref.restful import Works, Etiquette
# from scihub_blastoise import SciHub_HydroPump


### PART 1: Partial Run SumCetCet Analysis

We're back after Barça!

- The first order of business is to combine all partial fill SumCetCet run data to a single dataset to analyze.


In [7]:
# Sum:

with open("sum_pr_cr_results", "rb") as fp:
    sum_pr_cr_results = pickle.load(fp)

with open("sum_pr_sh_succ", "rb") as fp:
    sum_pr_sh_succ = pickle.load(fp)

with open("sum_pr_sh_fail", "rb") as fp:
    sum_pr_sh_fail = pickle.load(fp)

In [3]:
# Cet

with open("cet_pr_cr_results", "rb") as fp:
    cet_pr_cr_results = pickle.load(fp)

with open("cet_pr_sh_succ", "rb") as fp:
    cet_pr_sh_succ = pickle.load(fp)

with open("cet_pr_sh_fail", "rb") as fp:
    cet_pr_sh_fail = pickle.load(fp)

In [4]:
# Ogi:

with open("pr_cr_results", "rb") as fp:
    ogi_pr_cr_results = pickle.load(fp)

with open("pr_sh_succ", "rb") as fp:
    ogi_pr_sh_succ = pickle.load(fp)

with open("pr_sh_fail", "rb") as fp:
    ogi_pr_sh_fail = pickle.load(fp)

The initial analysis shows that around 50% of the metadata retr. from CR (pr_cr_results), was used for SH date extraction phase.

However, 40-50% of these articles actually contaion DATE info, therefore although the Req_Count is more than satisfied in the CR phase, SH results still do not satisfy for all journals.

Once the analysis is complete, we can use the pr_cr_results, to filter out the UNTESTED artcls, and pass them through the SH process.

In [46]:
sum_pr_cr_results = pd.DataFrame(sum_pr_cr_results)
sum_pr_sh_succ = pd.DataFrame(sum_pr_sh_succ)
sum_pr_sh_fail = pd.DataFrame(sum_pr_sh_fail)

sum_sh_results =  pd.concat([sum_pr_sh_succ, sum_pr_sh_fail])

sum_pr_results = pd.merge(sum_pr_cr_results, sum_sh_results, how="left", on="DOI")


In [47]:
cet_pr_cr_results = pd.DataFrame(cet_pr_cr_results)
cet_pr_sh_succ = pd.DataFrame(cet_pr_sh_succ)
cet_pr_sh_fail = pd.DataFrame(cet_pr_sh_fail)

cet_sh_results =  pd.concat([cet_pr_sh_succ, cet_pr_sh_fail])

cet_pr_results = pd.merge(cet_pr_cr_results, cet_sh_results, how="left", on="DOI")


In [48]:
ogi_pr_cr_results = pd.DataFrame(ogi_pr_cr_results)
ogi_pr_sh_succ = pd.DataFrame(ogi_pr_sh_succ)
ogi_pr_sh_fail = pd.DataFrame(ogi_pr_sh_fail)

ogi_sh_results =  pd.concat([ogi_pr_sh_succ, ogi_pr_sh_fail])

ogi_pr_results = pd.merge(ogi_pr_cr_results, ogi_sh_results, how="left", on="DOI")

Concatinating all partial run results to a single dataset: (pr_results_df)

In [54]:
pr_results_df = pd.concat([sum_pr_results, cet_pr_results,ogi_pr_results], ignore_index=True, verify_integrity=True)

In [59]:
pr_results_df.Dates.map(type).value_counts()

<class 'float'>    41840
<class 'str'>      35472
<class 'list'>     24708
Name: Dates, dtype: int64

Now that all the partial run results are consolidated:

TTD:
- Need to filter out the succesfull artcls, pass them through pre-processing, add them to q1_date_df

- Need to filter out FAILED, UNTESTED dfs


### PART 2: UPDATING q1_date_df

In [60]:
# TTD 1:
# Taken from 7c:

# 1.3 - Filtering to only successful artcls:
q1_date_pr = pr_results_df[pr_results_df.Dates.map(type)==list]
q1_date_pr.set_index("DOI", drop=True, inplace=True)

# 1.4 - Feature Eng. : 
def earlier_date_part(row):
    try:
        pp_year = row["published-print"]["date-parts"][0][0]
    except TypeError:
        pp_year = None

    try:   
        po_year = row["published-online"]["date-parts"][0][0]
    except TypeError:
        po_year = None

    if po_year is None:
        return row["published-print"]

    elif pp_year is None:
        return row["published-online"]

    else:
        if pp_year<po_year:
            return row["published-print"]
        elif po_year<pp_year:
            return row["published-online"]
        else:
            try:
                pp_month = row["published-print"]["date-parts"][0][1]
            except IndexError:
                pp_month = 12
            try:
                po_month = row["published-online"]["date-parts"][0][1]
            except IndexError:
                po_month = 12
            if pp_month<po_month:
                return row["published-print"]
            elif po_month<pp_month:
                return row["published-online"]
            else:
                return row["published-online"]


q1_date_pr.loc[:,"published"] = q1_date_pr.apply(earlier_date_part, axis=1)

q1_date_pr = q1_date_pr[["JRNL_ID", "ISSN", "issued", "container-title", "publisher", "title", "author", "subject", "references-count", "is-referenced-by-count", "published", "published-print", "published-online", "direct_url", "Dates", "Keywords"]]


<ipython-input-60-6f3939fe8a91>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1_date_pr.loc[:,"published"] = q1_date_pr.apply(earlier_date_part, axis=1)


__q1_date_pr__ is the succesful articles from partial run dataset:

However, it should be noted that some (also apparent in q1_date_df) only have "Accepted Date" and not "Received Date"

This MUST be dealt with later in this notebook!!!


In [ ]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

In [64]:
len(q1_date_df)

140993

In [65]:
q1_date_df = pd.concat([q1_date_df, q1_date_pr], verify_integrity=True)

__Fixing "only Accepted Date" issue:__

In [67]:
q1_date_df = q1_date_df[q1_date_df.Dates.map(lambda x: False if x[0]==None else True)]

SAVING & DUMPing all used datasets:

In [70]:
with open("q1_date_pr","wb") as p:
    pickle.dump(q1_date_pr, p)

In [121]:
# pr_results_df has ALL the extracted info from SumCetCet partial run

with open("pr_results_df","wb") as p:
    pickle.dump(pr_results_df, p)

In [69]:
with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

Currently q1_date_df has 156754 articles that are completely ready for analysis.

The remaining TDDs are:

__1- Article related:__
- Consolidate all UNTESTED article metadata -> only available from partial fill run dataset? - DONE
- Consolidate all FAIED articles to a single dataset -> final extraction, partial fill, q1_date_df_all, OTHERS? - DONE

__2- Journal related:__
- update q1_jrnl_df and q1_jrnl_expladed with the new information
- re-calc Req_Count & check if/how many UNTESTED can be used
- compare with pivots to check total necessary data

__3- Author related:__
- Finalize the author_df 
- Complete ScopusAPI Author Search run

__4- ANALYSIS & STATS & VISUALIZE__

__5- WRITE REPORT__

__6- FIN!__


### PART 3: UNTESTED AND FAILED Article Datasets

The datasets to be considered are:

- q1_date_df_all (Backups)
- q1_meta_fe
- q1_meta_fe_all
- date_df_full (5a)
- date_df_5b (5b)
- date_df_ok (Main Files)
- date_df_ok_final (Main Files)
- pr_results_df (7e)

To prevent to many variables each will be loaded & compared with q1_date_df and saved accordingly.


In [77]:
succ_doi_list = q1_date_df.index.tolist()

In [ ]:
# Code used for testing below:

# with open("date_df_ok_final", "rb") as fp:
#     testo = pickle.load(fp)

# temp_remn = testo[~testo.set_index("DOI").index.isin(succ_doi_list)].set_index("DOI")
# temp_remn = testo[~testo.index.isin(succ_doi_list)]

# remainder_df = pd.concat([remainder_df,q1_date_pr], verify_integrity=True)
# remainder_df = pd.concat([remainder_df,temp_remn])

__Results:__

- __q1_date_df_all (Backups)__
    - Total len: 36K
    - Not in q1_date_df: 4242
    - 

- __q1_meta_fe__
    - Total len: 138,184
    - Not in q1_date_df: 35,258
    - Already included : 131
    - 

- __q1_meta_fe_all__
    - Total len: 138,184
    - Not in q1_date_df: 35,258 
    - SKIPPED AS same as the previous df
    - 
- __date_df_full (5a)__
    - Total len: 128,374
    - Not in q1_date_df: 96,156
    - Already included: 4615
    - 
- __date_df_5b (5b)__
    - SKIPPED as pnly 36,514 articles with only SciHub information
    - 
- __date_df_ok (Main Files)__
    - Total len: 114,759
    - Not in q1_date_df: 82,686 
    - Already included: 82,773 SKIPPED as a result
- __date_df_ok_final (Main Files)__
    - Total len: 114,759
    - Not in q1_date_df: 82,686 
    - SKIPPED as same as previous
    - 
- __pr_results_df (7e)__
    - Total len:102,020
    - Not in q1_date_df: 78,645 
    - Already included: 4616
    - 


All the articles are concat'd into remainder_df. This df can be used to create FAILED & UNTESTED datasets

In [129]:
with open("remainder_df","wb") as p:
    pickle.dump(remainder_df, p)

#### PART 3.1: remainder_df:

In [2]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("remainder_df", "rb") as fp:
    remainder_df = pickle.load(fp)


In [5]:
remainder_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214301 entries, 10.1007/s10441-011-9133-1 to 10.1214/18-ejp208
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ISSN                    214301 non-null  object 
 1   issued                  214301 non-null  object 
 2   container-title         214301 non-null  object 
 3   publisher               214294 non-null  object 
 4   title                   214301 non-null  object 
 5   author                  192795 non-null  object 
 6   subject                 213933 non-null  object 
 7   references-count        118145 non-null  float64
 8   is-referenced-by-count  214301 non-null  int64  
 9   published               39116 non-null   object 
 10  published-print         201171 non-null  object 
 11  published-online        119803 non-null  object 
 12  direct_url              41047 non-null   object 
 13  Dates                   41047 non-null   obj

In [8]:
# Drop unnec. colms:
remainder_df.drop(["reference","language"], axis=1, inplace=True)

In [19]:
remainder_df[remainder_df.Results.notna()].Results.map(type).value_counts()

<class 'list'>    82686
<class 'str'>     13470
Name: Results, dtype: int64

In [22]:
remainder_df[(remainder_df.Results.notna() & (remainder_df.Results.map(type)==list))]["Results"].map(len).value_counts()

2    78240
3     4446
Name: Results, dtype: int64

In [13]:
remainder_df[(remainder_df.Dates.notna()) & (remainder_df.Results.notna()) ]

,ISSN,issued,container-title,publisher,title,author,subject,references-count,is-referenced-by-count,published,published-print,published-online,direct_url,Dates,Keywords,Affiliation,JRNL_ID,reference-count,Results
DOI,,,,,,,,,,,,,,,,,,,


In [12]:
remainder_df[(remainder_df["reference-count"].notna()) & (remainder_df["references-count"].notna()) ]

,ISSN,issued,container-title,publisher,title,author,subject,references-count,is-referenced-by-count,published,published-print,published-online,direct_url,Dates,Keywords,Affiliation,JRNL_ID,reference-count,Results
DOI,,,,,,,,,,,,,,,,,,,


__remainder_df__

__1- Succ:__
- has Dates & type == list
- has Results & len == 3

__2- Fail:__
- has Dates & type==str
- has Results & len==2

__3- Untested:__
- no Results & Dates cols.
- DOI not in any other df

In [24]:
# remainder_succ:

remainder_succ = remainder_df[(remainder_df.Dates.notna() & (remainder_df.Dates.map(type)==list))]

remainder_succ = pd.concat([remainder_succ, remainder_df.fillna("-")[(remainder_df.fillna("-").Results.notna() & (remainder_df.Results.fillna("-").map(len)==3))]])

In [33]:
# remainder_fail:

remainder_fail = remainder_df[(remainder_df.Dates.notna() & (remainder_df.Dates.map(type)==str))]

remainder_fail = pd.concat([remainder_fail, remainder_df.fillna("-")[(remainder_df.fillna("-").Results.notna()) & (remainder_df.Results.fillna("-").map(len)==2)]])

In [42]:
# remainder_untested:

# Need to drop all DOIs from other datasets

date_df_dois = q1_date_df.index.to_list()

rem_suc_dois = remainder_succ.index.tolist()

rem_fail_dois = remainder_fail.index.tolist()

In [43]:
all_dois = set(date_df_dois + rem_suc_dois + rem_fail_dois)

In [71]:
with open("all_dois","wb") as p:
    pickle.dump(all_dois, p)

In [45]:
remainder_untested = remainder_df[~remainder_df.index.isin(all_dois)]

In [69]:
remainder_untested.shape

(80243, 19)

In [52]:
remainder_untested.Results.value_counts()

article_not_in_Scihub    9454
skipped                  3655
cant_read_pdf             242
direct_url_error           10
pdf_bytes_error             7
article_not_in Scihub       2
Name: Results, dtype: int64

In [55]:
remainder_untested[remainder_untested.Results.isna()].shape

(76588, 19)

The result above shows that remainder_untested has 89,958 articles remaining. However, due to the filtering conditions some are actually failed articles & will be moved to remainder_fail dataset.

The rest can be used in the following steps if necessary.


It is also seen that this step increased the number of untested artcl. count from 41K (pr_results_df) to 80K.


In [61]:
remainder_fail = pd.concat([remainder_fail, remainder_untested[~(remainder_untested.Results.isna()) | (remainder_untested.Results == "skipped")]])

remainder_untested = remainder_untested[(remainder_untested.Results.isna()) | (remainder_untested.Results == "skipped")]

All the remainder datasets are ready for future steps, so each will be saved accordingly.

In [70]:
with open("remainder_df","wb") as p:
    pickle.dump(remainder_df, p)

with open("remainder_succ","wb") as p:
    pickle.dump(remainder_succ, p)

with open("remainder_fail","wb") as p:
    pickle.dump(remainder_fail, p)

with open("remainder_untested","wb") as p:
    pickle.dump(remainder_untested, p)

### PART 4: Journal Datasets


In [4]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

with open("q1_jrnl_expladed", "rb") as fp:
    q1_jrnl_expladed = pickle.load(fp)

with open("remainder_untested", "rb") as fp:
    remainder_untested = pickle.load(fp)

In [6]:
q1_jrnl_df.Required_Count.sum()

29921

In [7]:
q1_jrnl_df.Current_Count.sum()

140993

In [17]:
len(q1_date_df)

156754

In [19]:
q1_jrnl_df["New_Current_Count"] = q1_date_df.groupby("JRNL_ID")["Dates"].count()

In [22]:
len(q1_jrnl_df[q1_jrnl_df.New_Current_Count < q1_jrnl_df.Current_Count])

297

In [23]:
len(q1_jrnl_df[q1_jrnl_df.New_Current_Count > q1_jrnl_df.Current_Count])

9652

It is seen that for 297 journals, fixing the "Received Date" issue caused a decrease in the number of articles. 

Whereas 9652 journals now have more articles after the partial fill run!



In [24]:
q1_jrnl_df.drop("Current_Count", axis=1, inplace=True)

In [26]:
q1_jrnl_df.drop("Required_Count", axis=1, inplace=True)

In [28]:
q1_jrnl_df.rename(columns={"New_Current_Count":"Current_Count"}, inplace=True)

q1_jnrl_df is now updated & can be SAVEd.

In [30]:
with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

Will go back to notebook 7b to re-create variables. 

__NOTE:__ During the re-run of 7b, it is seen that there are 26 articles that were written in the 20th century, but was published online recently; causing them to appear in the dataset, although they were published much earlier. These will be dropped from the dataset in the 7b nb, BUT will not SAVE q1_date_df there to avoid confusion.

An example: q1_date_df.loc["10.1080/00263206.2010.527121","direct_url"]

Therefore, these articles need to be dropped & q1_date_df will be updated HERE! 

__NOTE 2:__ Also need to check other edge cases for articles written before 2010 & after 2020!

In [ ]:
# Code for the case above:

#q1_date_df[q1_date_df.Dates.map(lambda x: x[0][-4:]).map(lambda x: not x.startswith("20"))]

During the analysis it is also seen that the current articles in q1_date_df requires Subj. Areas assigned that will also in line with the matrixes!

This will be the main focus TOMORROW!

The best strategy seems to be saving the pivots from 5b to be usedin this nb.

In [2]:

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)


with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

### __TTD:__

- add a JRNL_YEAR column to q1_date_df - DONE

- __q1_jnrl_df:__ - DONE
    - fix Current_Count
    - update 9+1 CR_columns - DONE
    - as we had retrials of some FAILED and all PARTIAL OK jrnls, need to compare with Req_Count to update Consider column
    - as a final touch re-write the conditions in the loop for Consider column


- add a S_A column 

- create pivot for currently SUCCESSFULL article count

- calc. the necessary article number

- re-distr. (WITH S_A info) the necessary count to q1_jrnl_df based on consider column (New_Req_Count)

- final extraction!

In [6]:
# TTD 1:

jrnl_year_dict = q1_jrnl_df.Year.to_dict()

q1_date_df["JRNL_YEAR"] = q1_date_df.JRNL_ID.map(jrnl_year_dict)

In [47]:
# TTD 1 - jrnl:

q1_jrnl_df.Current_Count = q1_jrnl_df.Current_Count.fillna(0)
q1_jrnl_df.Current_Count = q1_jrnl_df.Current_Count.astype(int)

ID
1_0        13
1_1         7
1_2         8
1_3        10
1_4         5
           ..
1_66911     3
1_66912     3
1_66913     0
1_66914     8
1_66915     0
Name: Current_Count, Length: 66916, dtype: int32

In [12]:
# TTD 2:
# Need to find the  most up-to-date Req. Count column:

# with open("q1_jrnl_df_25may", "rb") as fp:
#     q1_jrnl_df_25may = pickle.load(fp)

with open("q1_jrnl_df_21july", "rb") as fp:
    q1_jrnl_df_21july = pickle.load(fp)

q1_jrnl_df = q1_jrnl_df.merge(q1_jrnl_df_21july["Required_Count"], how="left", left_index=True, right_index=True)


In [52]:
# Also added the "old" Current_Count column to analyze the differences:
q1_jrnl_df_21july.rename(columns={"Current_Count":"Old_C_Count"},inplace=True)
q1_jrnl_df = q1_jrnl_df.merge(q1_jrnl_df_21july["Old_C_Count"], how="left", left_index=True, right_index=True)


In [57]:
len(q1_jrnl_df[q1_jrnl_df.Old_C_Count > q1_jrnl_df.Current_Count])

1939

In [58]:
len(q1_jrnl_df[(q1_jrnl_df.Old_C_Count > 0) & (q1_jrnl_df.Current_Count == 0)])

1642

It is seen that during the RECEIVED DATE only update (removed articles if only had Acc. Date in q1_date_df), 1939 jrnls in total seen a decrease in numbers. Out of which, 1642 does not have any remaining jrnls left.

They can be tested again to be sure, but will be skipped due to time limitations.



In [31]:
# Updating 9 + 1:
for id in ["1_18052","1_24838","1_24839","1_24840","1_24841","1_24842","1_24844","1_66638","1_66639"]:
    q1_jrnl_df.loc[id,"CR_Count"] = 0

q1_jrnl_df.loc["1_24843","CR_Count"] = 1


__Updating "Consider":__

- Consider is more up-to-date than Req_count
- Therefore if already Consider= False, it was skipped in partial fill & failed one of the conditions of Consider update in part 7c
- So need to update Consider == True & Req_Count > 0 articles! 

---


- All the analysis above shows that in the current q1_jrnl_df:
    - if a jrnl has Current_Count > 0, it can be considered for the last run.
    - However, do not forget that there are some Consider= False BUT Cur_Count > 0 jrnls. These are fine and will be considered in the pivot and not in the Consider.
    - 
    
- All in all we will be left with 32,734 jrnls that have at least 1 article in q1_date_df 
- And 32,688 jrnls that can be CONSIDERed in the next fillings!  


In [65]:
len(q1_jrnl_df[q1_jrnl_df.Consider == True])

34380

In [61]:
len(q1_jrnl_df[(q1_jrnl_df.Current_Count > 0)])

32734

In [62]:
len(q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Current_Count > 0)])

32688

In [63]:
len(q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Current_Count== 0)])

1692

__NOTE:__ 
1_42139, 1_14566 these journals USED TO have articles in the dataset, BUT 0 after RECEIVED_DATE ONLY update!

Code: q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Current_Count== 0) & (q1_jrnl_df.Old_C_Count > 0)].sort_values("Old_C_Count", ascending=False)


In [68]:
# The update code: 

q1_jrnl_df.loc[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Current_Count > 0), "New_Consider"] = True

In [72]:
# Finalize the dataset:

q1_jrnl_df.drop(["Consider","Required_Count","Old_C_Count"], axis=1, inplace= True)
q1_jrnl_df.rename(columns={"New_Consider": "Consider"}, inplace=True)

Consider col. is now updated. Both datasets should be up-to-date and ready for future proessing.

In [76]:
with open("q1_jrnl_df","wb") as fp:
   pickle.dump(q1_jrnl_df, fp)

In [75]:
q1_jrnl_df.head()

,ISSN,Year,Title,Sbj_Area,Total_Docs,Country,Region,Publisher,Coverage,SJR,...,Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,CR_Count,SH_Results,Current_Count,Consider
ID,,,,,,,,,,,,,,,,,,,,,
1_0,"(00011541, 15475905)",2013,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",422,United States,Northern America,Wiley-Blackwell,1955-2020,"1,053",...,956,17490,2697,928,"2,82","41,45",343,True,13,True
1_1,"(00011541, 15475905)",2014,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",359,United States,Northern America,Wiley-Blackwell,1955-2020,"1,066",...,1084,15256,3359,1058,"3,16","42,50",376,True,7,True
1_2,"(00011541, 15475905)",2019,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",335,United States,Northern America,Wiley-Blackwell,1955-2020,"0,947",...,1203,15387,4649,1189,"3,70","45,93",384,True,8,True
1_3,"(00011541, 15475905)",2018,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",379,United States,Northern America,Wiley-Blackwell,1955-2020,"0,978",...,1195,17900,4436,1181,"3,71","47,23",366,True,10,True
1_4,"(00011541, 15475905)",2020,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",374,United States,Northern America,Wiley-Blackwell,1955-2020,"0,958",...,1171,18063,4720,1155,"4,01","48,30",337,True,5,True


### Part 5: Adding S_A to q1_date_df


In [79]:
q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Sbj_Area.map(len) == 1)].Current_Count.sum()

1    18549
2    10249
3     2871
4      796
5      174
6       47
7        2
Name: Sbj_Area, dtype: int64

In all posssible methods, we need to add the JRNL_S_A col. to q1_date_df:

In [91]:
jrnl_s_a_dict = q1_jrnl_df.Sbj_Area.map(lambda x: list(x.keys())).to_dict()

q1_date_df["JRNL_S_A"] = q1_date_df.JRNL_ID.map(jrnl_s_a_dict)

In [104]:
q1_date_df.JRNL_S_A.map(len).value_counts().sort_index()

1    56931
2    60616
3    25899
4     9986
5     2494
6      791
7       37
Name: JRNL_S_A, dtype: int64

In [106]:
def one_picker(sa_list):
    if len(sa_list) == 1:
        return sa_list[0]
    else:
        return np.nan

q1_date_df["S_A"] = q1_date_df.JRNL_S_A.map(one_picker)

In [117]:
pd.pivot_table(data=q1_date_df, columns="JRNL_YEAR", index="S_A", values="title", aggfunc="count")

JRNL_YEAR,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
S_A,,,,,,,,,,,
Agricultural and Biological Sciences,301,281,291,301,335,308,327,338,339,327,345
Arts and Humanities,189,192,215,202,271,211,247,252,232,216,243
"Biochemistry, Genetics and Molecular Biology",184,182,169,168,182,186,185,151,165,167,156
"Business, Management and Accounting",176,172,209,199,204,235,186,175,201,216,188
Chemical Engineering,102,75,85,81,113,73,109,78,100,107,74
Chemistry,128,135,139,162,154,168,155,134,122,131,131
Computer Science,188,183,181,171,168,206,188,185,189,166,168
Decision Sciences,6,8,10,7,3,7,12,7,7,7,10
Dentistry,226,169,264,224,190,240,308,255,358,318,310


In [118]:
with open("7b_\\q1_a_ss", "rb") as fp:
    q1_a_ss = pickle.load(fp)

The best way to assign S_A to q1_date_df would be to calculate the total number of eligible artlcs. for each cell ( so, creating a new pivot from exploded JRNL_S_A column) and dividing that with q1_a_ss to calculate the "S_A probability". 

#### Percentage Calculation:

In [125]:
q1_date_exploded = q1_date_df.explode("JRNL_S_A")

In [130]:
pd.pivot_table(data=q1_date_exploded, columns="JRNL_YEAR", index="JRNL_S_A", values="title", aggfunc="count")

JRNL_YEAR,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
JRNL_S_A,,,,,,,,,,,
Agricultural and Biological Sciences,1331,1378,1375,1375,1324,1370,1338,1313,1325,1266,1262
Arts and Humanities,873,887,903,839,875,888,869,862,876,909,836
"Biochemistry, Genetics and Molecular Biology",1506,1533,1508,1510,1510,1465,1423,1357,1318,1301,1285
"Business, Management and Accounting",932,957,948,997,947,988,1051,1068,1046,1103,1108
Chemical Engineering,1069,1124,1083,1050,1034,1060,991,966,984,1000,1004
Chemistry,1218,1253,1201,1213,1195,1150,1094,986,1008,1067,1127
Computer Science,1214,1238,1208,1186,1201,1226,1238,1204,1239,1286,1269
Decision Sciences,711,743,732,719,679,703,732,762,739,773,759
Dentistry,389,351,384,383,275,376,471,390,446,479,424


In [135]:
perc_pivot = (q1_a_ss / pd.pivot_table(data=q1_date_exploded, columns="JRNL_YEAR", index="JRNL_S_A", values="title", aggfunc="count")).round(3)

In [138]:
q1_date_exploded.drop("S_A", axis=1, inplace=True)

In [156]:
def perc_assign(row):
    # year = row["JRNL_YEAR"]
    # s_a = row["JRNL_S_A"]
    return perc_pivot.loc[row["JRNL_S_A"],row["JRNL_YEAR"]]
    

In [161]:
q1_date_exploded["Perc"] = q1_date_exploded.apply(perc_assign, axis=1)

In [168]:
q1_date_df["Perc_SA"] = q1_date_exploded.groupby(q1_date_exploded.index)["JRNL_S_A"].apply(list)

In [171]:
q1_date_df["Perc_Percs"] = q1_date_exploded.groupby(q1_date_exploded.index)["Perc"].apply(list)

q1_date_df["Perc_Percs"] = q1_date_df.Perc_Percs.map(lambda x: [round((item/sum(x)),3) for item in x])

In [183]:
q1_date_df["S_A_Percs"] = q1_date_df.apply(lambda x: dict(zip(x.Perc_SA, x.Perc_Percs)), axis=1)

In [185]:
q1_date_df["Chosen_S_A"] = q1_date_df.S_A_Percs.map(lambda x: random.choices(list(x.keys()), weights=x.values(), k=1))

q1_date_df["Chosen_S_A"] = q1_date_df.Chosen_S_A.map(lambda x: x[0])

In [191]:
sa_chosen_pivot = pd.pivot_table(data=q1_date_df, columns="JRNL_YEAR", index="Chosen_S_A", values="title", aggfunc="count")

In [195]:
# New Req_Count total:

(q1_a_ss - sa_chosen_pivot).applymap(lambda x: 0 if x<0 else x).sum().sum()

21856

Will stop here for now! 

Will assume the S_A assigned to be TRUE!

The total number of necessary articles increased from around 15K for appr. 22K.

This is OK as the total number of articles will be somewhere around 180K, which was the plan before assignment.

Will save all variables

Tomorrow:

- will use sa_chosen_pivot to calc. Req_Count for the 180K run
- will save S_A this time beforehand
- arrange the 180K SumCetCet run 
- send codes!

- start working on author_df!

In [196]:
with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

with open("q1_date_exploded","wb") as p:
    pickle.dump(q1_date_exploded, p)